In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
dataset = pd.read_csv('house.csv')
dataset.drop(['Id'],
             axis=1,
             inplace=True)

dataset['SalePrice'] = dataset['SalePrice'].fillna(
  dataset['SalePrice'].mean())


new_dataset = dataset.dropna()
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
OH_encoder = OneHotEncoder(sparse_output=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

X = df_final.drop('SalePrice', axis=1)  # Specify the column name for the target variable
y = df_final['SalePrice']  # Specify the column name for the target variable

# Split the data into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)




NameError: name 'OneHotEncoder' is not defined

In [16]:
dataset = pd.read_csv('house.csv')
dataset['BldgType'].dtype == 'category'

False

2

In [18]:
trainy['new'] = pd.factorize(trainy[0])[0]

In [26]:
map = {index: value for index, value in enumerate(trainy.unique())}

In [73]:
def relu(x):
    return np.where(x > 0, x, 0)
def drelu(x):
    return np.where(x > 0, 1,0)


In [74]:
class NeuralNetwork:
    def __init__(self,hidden = []) -> None:
        self.sizes = hidden
        self.biases = None
        self.weights = None
        self.type = None
        self.map = {}
    # detects what type of model we want and constructs our bias,weights,type and map dependent on the dataframe and series given
    def construct(self,X,Y):
        inputsize = X.shape[1]
        self.sizes.insert(0,inputsize)
        # Checking if its categorical or regression NN
        if Y.dtype == 'float64':
            self.sizes.append(1)
            self.type = 'regression'
        else:
            unique_values = len(Y.unique())
            self.sizes.append(unique_values)
            self.type = 'classification'
            self.map = {index: value for index, value in enumerate(Y.unique())}
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(self.sizes[:-1],self.sizes[1:])]

    # Helper for train function
    # Feeds our input into our neural network and determines the loss 
    def forward(self,input,answers):
        ## Input is supposed to be the training matrix
        input = input.T
        for b, w in zip(self.biases, self.weights):
            print(input.shape)
            dot = np.dot(w, input)+b
            
            input = relu(np.dot(w, input)+b)
        # input is now the output activation
        # If the type is regression, then the length of the output activation should be 1. 
        if self.type == 'regression':
            loss = ((answers - input) **2)/len(input)
        elif self.type == 'classification':
            input = input.T
            currentsum = 0
            for row,index in zip(input,)
                
            return
        return  
    
    # Returns a tuple of dw,db, which are layer by layer arrays that represent partials
    def backprop(self,inputx,inputy):
        # input x is a 1d vector ,input y is a scalar
        partialb = [np.zeros(b.shape) for b in self.biases]
        partialw = [np.zeros(w.shape) for w in self.weights]
        storedsums = []
        storedactivations = [inputx]
        

        # Forward propagation, storing the su
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            storedsums.append(z)
            activation = relu(z)
            storedactivations.append(activation)

        ## Actual backpropagation
        #First Layer
        ## Regression case. Thus, the activation for our output for our output layer is linear. Our loss function is mean square error, thus delta (dL/dz) is -2(y-z).
        if self.type == 'regression':
            dz = -2 * (y - activation[-1]).reshape(1,1)
            assert dz.shape == partialb[-1].shape
            dw = np.dot(dz, storedactivations[-2].T)
            assert dw.shape == partialw[-1].shape
            partialw[-1]= dw
        elif self.type == 'classification':
            print('test')


        # Finding partials for the rest of the layers
        for i in range(2,len(self.sizes)): 
            dz = drelu(storedsums[-i])
            delta = np.dot(self.weights[-i + 1],partialb[-i + 1]) * dz
            assert delta.shape == partialb[-i].shape
            partialb[-i] = delta   
            weight = np.dot(delta, storedactivations[-i - 1].T)
            assert weight.shape == partialw[-i].shape
            partialw[-i] = weight
        return (partialw,partialb)

    # Updates our weights and biases with a given batch
    def updatebatch(self,batchx,batchy,learningrate = 0.05):
        ## Assumptions these are numpy arrays
        currentsumb = [np.zeros(b.shape) for b in self.biases]
        currentsumw = [np.zeros(w.shape) for w in self.weights]
        for x,y in zip(batchx,batchy):
            gradientw,gradientb = self.backprop(x,y)
            for i in range(len(currentsumb)):
                currentsumb[i] = (gradientb[i] + currentsumb[i])
                currentsumw[i] = (gradientw[i] + currentsumw[i])
        
        for i in range(len(currentsumb)):
            self.weights[i] = self.weights[i] - (learningrate/len(batchy)) * currentsumw[i]
            self.weights[i] = self.biases[i] - (learningrate/len(batchy)) * currentsumb[i]
            
    def train(self,X,Y,batchsize = 32,epoch = 5):
        assert isinstance(X, pd.DataFrame) 
        assert isinstance(Y,pd.Series)
        self.construct(X,Y)
        
        Y = pd.factorize(Y)[0]
        self.forward(X,Y)
        minibatchesX = np.array_split(X, len(X) // batchsize)
        minibatchesY = np.array_split(Y, len(Y) // batchsize)
        #split dataframe into batches of size batchsize
        for i in range(epoch):
            for x,y in zip(minibatchesX,minibatchesY):
                print('test')


In [75]:
from sklearn.datasets import make_moons, make_blobs

X, y = make_moons(n_samples=100, noise=0.1)
y = y*2 - 1 # make y be -1 or1

trainx = pd.DataFrame(X)
trainy = pd.Series(y)

In [76]:
nn = NeuralNetwork([10,20])
nn.train(trainx,trainy)

[2, 10, 20, 2]
(2, 100)
(10, 100)
(20, 100)
[6.83053443 1.30872693]
[41.50068278 10.16084596]
[11.01733029  4.73667753]
[7.10791862 1.44725316]
[16.35469071  3.84433628]
[7.44300468 1.51741639]
[3.51523865 0.        ]
[17.92494857  7.30904948]
[27.20288569  9.11834385]
[48.43873059 12.43484923]
[16.56910051  3.14813534]
[11.96817595  5.12922416]
[10.23170785  3.87202336]
[8.85195705 2.34852841]
[51.25356229 12.84857902]
[10.64794132  4.53724656]
[28.49869247  8.50054641]
[22.6529093   9.00649723]
[8.42866244 3.9435803 ]
[9.18283725 4.01589708]
[22.1614716   8.82033009]
[31.58556289  6.73450357]
[4.18297552 0.        ]
[8.81656435 2.33061432]
[27.19330726  5.72408964]
[13.04026917  2.81894411]
[27.71013921  7.42413755]
[18.27654894  4.24495194]
[29.21679484  7.64066494]
[11.68134881  5.01677843]
[23.88929569  9.36426039]
[25.97567002  8.34761495]
[44.66873432 11.00088506]
[8.66052851 3.86755067]
[7.634255   2.64078046]
[12.66356946  5.29607106]
[26.93886128  9.39795475]
[37.05709667  8.